In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math


In [4]:
import random
import numpy as np
from numpy import ndarray

In [5]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 6 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [6]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/hepmass_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [7]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (SB, SQ), (MB, SQ), (MB, SQ), (SB, MQ), (MB, LQ), (MB, SQ), (SB, MQ),
     (SB, SQ), (MB, MQ), (LB, LQ), (SB, MQ), (MB, SQ), (MB, MQ), (MB, MQ),
      (MB, MQ), (SB, LQ), (SB, SQ), (MB, SQ), (SB, SQ), (MB, MQ)
 ]

In [20]:
# Compute total number of required points across all test combinations
required_points = sum(data_size for data_size, _ in test_combinations)

# Sample from the full dataset without replacement
if required_points > total_points:
    raise ValueError(f"Need {required_points} points but only {total_points} available.")

sampled_indices = np.random.choice(total_points, size=required_points, replace=False)
combined_dataset = dataset[sampled_indices]


In [21]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


In [ ]:
print(len(queries_list))
print(len(combined_dataset))

20
2180000


**-------------Cover Tree---------------**

**Izbicki's implementation**

In [ ]:
import numpy as np
from typing import Optional, List, Tuple
from dataclasses import dataclass

@dataclass
class Node:
    point: np.ndarray
    level: int
    children: List['Node']
    _maxdist_cache: Optional[float] = None  # Cache for maxdist

    @property
    def covdist(self) -> float:
        """Coverage distance of the node."""
        return 2.0 ** self.level

    @property
    def maxdist(self) -> float:
        """Maximum distance within the subtree rooted at this node."""
        if self._maxdist_cache is None:
            if not self.children:
                self._maxdist_cache = 0
            else:
                self._maxdist_cache = max(
                    self.covdist,
                    max(child.maxdist + np.linalg.norm(self.point - child.point)
                        for child in self.children)
                )
        return self._maxdist_cache

    def update_maxdist(self):
        """Reset and recompute maxdist cache."""
        self._maxdist_cache = None
        _ = self.maxdist  # Force recomputation

class CoverTree:
    def __init__(self):
        self.root: Optional[Node] = None

    def insert(self, x: np.ndarray) -> None:
        """Insert a point into the cover tree."""
        if self.root is None:
            self.root = Node(x, 0, [])
            return

        d = np.linalg.norm(self.root.point - x)
        if d > self.root.covdist:
            # Precompute levels to expand
            levels_to_expand = int(np.ceil(np.log2(d / self.root.covdist)))
            for _ in range(levels_to_expand):
                self.root = Node(self.root.point, self.root.level + 1, [self.root])
            self.root = Node(x, self.root.level + 1, [self.root])
            return

        self._insert_recursive(self.root, x)

    def _insert_recursive(self, p: Node, x: np.ndarray) -> bool:
        """Recursive insertion into the cover tree."""
        for child in p.children:
            if np.linalg.norm(child.point - x) <= child.covdist:
                if self._insert_recursive(child, x):
                    p.update_maxdist()
                    return True

        # If x fits within the current node's coverage
        if np.linalg.norm(p.point - x) <= p.covdist:
            p.children.append(Node(x, p.level - 1, []))
            p.update_maxdist()
            return True

        return False

    def find_nearest(self, x: np.ndarray) -> Tuple[np.ndarray, float]:
        """Find the nearest neighbor to the query point."""
        if self.root is None:
            raise ValueError("Tree is empty")

        return self._find_nearest_recursive(self.root, x, self.root)

    def _find_nearest_recursive(self, p: Node, x: np.ndarray, best: Node) -> Tuple[np.ndarray, float]:
        """Recursive nearest neighbor search with pruning."""
        d_p = np.linalg.norm(p.point - x)
        d_best = np.linalg.norm(best.point - x)

        # Update the best point if the current node is closer
        if d_p < d_best:
            best = p
            d_best = d_p

        # Sort children by distance to the query point
        for child in sorted(p.children, key=lambda c: np.linalg.norm(c.point - x)):
            d_x_to_child = np.linalg.norm(child.point - x)

            # Prune subtrees based on the pruning condition
            if d_best > d_x_to_child - child.maxdist:
                child_point, child_dist = self._find_nearest_recursive(child, x, best)
                if child_dist < d_best:
                    best = Node(child_point, p.level, [])
                    d_best = child_dist

        return best.point, d_best


-----------

**----------CV Run 1----------------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 133.66it/s]


Build Time: 8.44 sec, Search Time: 0.15 sec

1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 37.08it/s]


Build Time: 120.51 sec, Search Time: 0.54 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 246.17 sec, Search Time: 2.01 sec

18: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:01<00:00, 10.61it/s]


Build Time: 24.35 sec, Search Time: 1.88 sec

19: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:09<00:00, 10.36it/s]

Build Time: 246.70 sec, Search Time: 9.60 sec

=== Summary of CV Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 8.44 sec, Search: 0.15 sec
Data:  100000 points, Queries:   20 => Build: 120.51 sec, Search: 0.54 sec
Data:  100000 points, Queries:   20 => Build: 151.51 sec, Search: 0.84 sec
Data:   10000 points, Queries:  100 => Build: 16.17 sec, Search: 3.88 sec
Data:  100000 points, Queries:  500 => Build: 166.62 sec, Search: 22.10 sec
Data:  100000 points, Queries:   20 => Build: 177.94 sec, Search: 1.00 sec
Data:   10000 points, Queries:  100 => Build: 18.46 sec, Search: 5.04 sec
Data:   10000 points, Queries:   20 => Build: 18.23 sec, Search: 1.04 sec
Data:  100000 points, Queries:  100 => Build: 190.99 sec, Search: 5.64 sec
Data: 1000000 points, Queries:  500 => Build: 2170.05 sec, Search: 41.44 sec
Data:   10000 points, Queries:  100 => Build: 23.09 sec, Search: 8.27 sec
Data:  100000 points, Queries:   20 => Build: 235.58 sec, Search: 1.88 sec
Data:  100000 points, Q

**----------CV Run 2-----------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
# Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 143.11it/s]


Build Time: 9.20 sec, Search Time: 0.14 sec

1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 30.93it/s]


Build Time: 118.37 sec, Search Time: 0.64 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 245.22 sec, Search Time: 2.12 sec

18: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:02<00:00,  9.85it/s]


Build Time: 24.62 sec, Search Time: 2.02 sec

19: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:10<00:00,  9.96it/s]

Build Time: 249.15 sec, Search Time: 9.98 sec

=== Summary of CV Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 9.20 sec, Search: 0.14 sec
Data:  100000 points, Queries:   20 => Build: 118.37 sec, Search: 0.64 sec
Data:  100000 points, Queries:   20 => Build: 151.70 sec, Search: 0.77 sec
Data:   10000 points, Queries:  100 => Build: 16.16 sec, Search: 4.14 sec
Data:  100000 points, Queries:  500 => Build: 167.49 sec, Search: 23.58 sec
Data:  100000 points, Queries:   20 => Build: 177.52 sec, Search: 1.01 sec
Data:   10000 points, Queries:  100 => Build: 18.10 sec, Search: 5.47 sec
Data:   10000 points, Queries:   20 => Build: 18.42 sec, Search: 1.14 sec
Data:  100000 points, Queries:  100 => Build: 192.74 sec, Search: 6.29 sec
Data: 1000000 points, Queries:  500 => Build: 2242.17 sec, Search: 45.25 sec
Data:   10000 points, Queries:  100 => Build: 23.96 sec, Search: 8.28 sec
Data:  100000 points, Queries:   20 => Build: 237.62 sec, Search: 1.97 sec
Data:  100000 points, Q

**---------CV Analysis-------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 4757.09',
    'Total Time: 4863.04'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 4810.0650 sec, Std Dev: 74.9180 sec, CV: 1.56%


--------------

**--------------------------VP_MWV-----------------------------**

In [ ]:
import numpy as np

class VPNode:
    def __init__(self, vp=None):
        # Vantage point
        self.vp = vp

        # Distance bounds
        self.low_near = float('inf')
        self.low_far = 0
        self.high_near = float('inf')
        self.high_far = 0

        # Child nodes
        self.lowChild = None
        self.highChild = None

        # Bucket for leaf nodes
        self.bucket = []

    def is_leaf(self):
        return self.lowChild is None and self.highChild is None

class VPTree:
    def __init__(self, bucket_size=50, distance_fn=None):
        """
        Initialize a VP-Tree

        Parameters:
        - bucket_size: Square root of maximum bucket size (b where b² is max bucket size)
        - distance_fn: Distance function to use (defaults to Euclidean)
        """
        self.bucket_size = bucket_size
        self.root = None

        # Default to Euclidean distance if none provided
        self.distance_fn = distance_fn if distance_fn is not None else self._euclidean_distance

    def _euclidean_distance(self, x, y):
        """Calculate Euclidean distance between two points"""
        x = np.asarray(x)
        y = np.asarray(y)
        diff = x - y
        return np.sqrt(np.dot(diff, diff))

    def insert(self, x):
        """Insert a new datapoint x into the VP-tree"""
        # If tree is empty, create root with this point as VP
        if self.root is None:
            self.root = VPNode(vp=x)
            return

        p = self.root

        # Traverse until we reach a leaf node
        while not p.is_leaf():
            # Calculate distance from x to current vantage point
            dist = self.distance_fn(x, p.vp)

            # Determine which child to go to
            if dist < (p.low_far + p.high_near) / 2:
                # Update distance bounds
                p.low_far = max(dist, p.low_far)
                p.low_near = min(dist, p.low_near)

                # Go to low (near) child
                p = p.lowChild
            else:
                # Update distance bounds
                p.high_far = max(dist, p.high_far)
                p.high_near = min(dist, p.high_near)

                # Go to high (far) child
                p = p.highChild

        # Add x to bucket leaf node p
        p.bucket.append(x)

        # If bucket exceeds b², split it
        if len(p.bucket) > self.bucket_size**2:
            self._split_node(p)

    def _select_vantage_point(self, points):
        """Select a vantage point from the bucket
        Can use various methods; simple random selection is shown here.
        """
        # For simplicity, using the first point as VP
        # Could implement more sophisticated selection methods
        if points:
            return points[0]
        return None

    def _split_node(self, p):
        """Split a node using the CART-inspired weighted variance method"""
        points = p.bucket.copy()

        # Select vantage point from bucket
        vp = self._select_vantage_point(points)
        points.remove(vp)  # Remove VP from points list

        # Calculate distances to vantage point
        distances = [(self.distance_fn(vp, point), point) for point in points]

        # Sort by distance
        distances.sort(key=lambda x: x[0])
        sorted_points = [point for _, point in distances]
        sorted_dists = [dist for dist, _ in distances]

        # Find optimal split using weighted variance minimization
        n = len(sorted_points)
        best_split = 0
        min_weighted_variance = float('inf')

        for s in range(1, n):
            # Calculate variances for 1:s and s:n
            if s > 0:
                var_left = np.var(sorted_dists[:s]) if s > 1 else 0
            else:
                var_left = 0

            if n-s > 0:
                var_right = np.var(sorted_dists[s:]) if n-s > 1 else 0
            else:
                var_right = 0

            # Calculate weighted variance
            weighted_var = s * var_left + (n-s) * var_right

            if weighted_var < min_weighted_variance:
                min_weighted_variance = weighted_var
                best_split = s

        # Create new children nodes
        p.lowChild = VPNode(vp=vp)
        p.highChild = VPNode(vp=self._select_vantage_point(sorted_points[best_split:]))

        # Distribute points to children based on best split
        p.lowChild.bucket = sorted_points[:best_split]
        p.highChild.bucket = sorted_points[best_split:]

        # Initialize distance bounds
        if p.lowChild.bucket:
            low_dists = sorted_dists[:best_split]
            p.low_near = min(low_dists)
            p.low_far = max(low_dists)

        if p.highChild.bucket:
            # If we have a vantage point for the high child, calculate distances to it
            high_dists = [self.distance_fn(p.highChild.vp, point) for point in p.highChild.bucket]
            p.high_near = min(high_dists) if high_dists else float('inf')
            p.high_far = max(high_dists) if high_dists else 0

        # Clear the bucket as it's no longer a leaf
        p.bucket = []

    def nearest_neighbor(self, query):
        """Optimized nearest neighbor search with batch processing for bucket points"""
        best_dist = float('inf')
        best_point = None

        # Use a stack instead of recursion to reduce overhead
        # Each entry is (node, min_possible_dist)
        stack = [(self.root, 0)]

        while stack:
            node, min_dist = stack.pop()

            # Skip this node if it cannot contain a better solution
            if min_dist >= best_dist:
                continue

            # Check vantage point distance
            if node.vp is not None:
                # Calculate distance to vantage point
                diff = np.subtract(query, node.vp)
                dist_to_vp = np.sqrt(np.dot(diff, diff))

                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.vp

            # If leaf node, check all points in bucket using batch processing
            if node.is_leaf():
                if node.bucket:
                    # Convert bucket to numpy array once
                    bucket_array = np.array(node.bucket)

                    # Vectorized distance calculation
                    query_array = np.array(query)
                    diffs = bucket_array - query_array
                    squared_dists = np.sum(diffs**2, axis=1)

                    # Find minimum distance point without unnecessary sqrt operations
                    min_idx = np.argmin(squared_dists)
                    min_squared_dist = squared_dists[min_idx]

                    # Only compute sqrt for potentially better distance
                    if min_squared_dist < best_dist**2:
                        min_dist = np.sqrt(min_squared_dist)
                        if min_dist < best_dist:
                            best_dist = min_dist
                            best_point = node.bucket[min_idx]
                continue

            # Calculate threshold
            threshold = (node.low_far + node.high_near) / 2

            # Calculate bounds for both children
            if dist_to_vp < threshold:
                # Query is in near region
                # Add far child to stack first (so near child gets processed first)
                if node.highChild:
                    far_bound = max(0, dist_to_vp - node.low_far)
                    if far_bound < best_dist:
                        stack.append((node.highChild, far_bound))

                # Add near child with zero bound (must contain query)
                if node.lowChild:
                    stack.append((node.lowChild, 0))
            else:
                # Query is in far region
                # Add near child to stack first (so far child gets processed first)
                if node.lowChild:
                    near_bound = max(0, node.high_near - dist_to_vp)
                    if near_bound < best_dist:
                        stack.append((node.lowChild, near_bound))

                # Add far child with zero bound (must contain query)
                if node.highChild:
                    stack.append((node.highChild, 0))

        return best_dist, best_point

-----------

**VP_MWV (Run 1)**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 610.08it/s]


Build Time: 2.05 sec, Search Time: 0.03 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 323.19it/s]


Build Time: 24.31 sec, Search Time: 0.06 sec

3: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 31.08 sec, Search Time: 0.05 sec

19: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 521.61it/s]


Build Time: 3.58 sec, Search Time: 0.04 sec

20: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 548.45it/s]

Build Time: 35.52 sec, Search Time: 0.18 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 2.05 sec, Search: 0.03 sec
Data:  100000 points, Queries:   20 => Build: 24.31 sec, Search: 0.06 sec
Data:  100000 points, Queries:   20 => Build: 25.99 sec, Search: 0.03 sec
Data:   10000 points, Queries:  100 => Build: 2.29 sec, Search: 0.11 sec
Data:  100000 points, Queries:  500 => Build: 26.73 sec, Search: 0.87 sec
Data:  100000 points, Queries:   20 => Build: 29.74 sec, Search: 0.03 sec
Data:   10000 points, Queries:  100 => Build: 3.37 sec, Search: 0.12 sec
Data:   10000 points, Queries:   20 => Build: 1.78 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 27.46 sec, Search: 0.14 sec
Data: 1000000 points, Queries:  500 => Build: 301.17 sec, Search: 2.05 sec
Data:   10000 points, Queries:  100 => Build: 3.41 sec, Search: 0.16 sec
Data:  100000 points, Queries:   20 => Build: 33.87 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => B

**-----VP_MWV Run 2-------------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 696.56it/s]


Build Time: 2.89 sec, Search Time: 0.03 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 169.65it/s]


Build Time: 23.75 sec, Search Time: 0.12 sec

3: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 34.21 sec, Search Time: 0.05 sec

19: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 410.73it/s]


Build Time: 2.85 sec, Search Time: 0.05 sec

20: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 542.37it/s]

Build Time: 29.35 sec, Search Time: 0.18 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 2.89 sec, Search: 0.03 sec
Data:  100000 points, Queries:   20 => Build: 23.75 sec, Search: 0.12 sec
Data:  100000 points, Queries:   20 => Build: 25.10 sec, Search: 0.04 sec
Data:   10000 points, Queries:  100 => Build: 3.04 sec, Search: 0.12 sec
Data:  100000 points, Queries:  500 => Build: 29.53 sec, Search: 0.98 sec
Data:  100000 points, Queries:   20 => Build: 27.95 sec, Search: 0.04 sec
Data:   10000 points, Queries:  100 => Build: 2.95 sec, Search: 0.14 sec
Data:   10000 points, Queries:   20 => Build: 2.83 sec, Search: 0.04 sec
Data:  100000 points, Queries:  100 => Build: 32.37 sec, Search: 0.18 sec
Data: 1000000 points, Queries:  500 => Build: 306.17 sec, Search: 2.36 sec
Data:   10000 points, Queries:  100 => Build: 3.46 sec, Search: 0.18 sec
Data:  100000 points, Queries:   20 => Build: 31.87 sec, Search: 0.05 sec
Data:  100000 points, Queries:  100 => B

**-------VP_MWV Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 372.76it/s]


Build Time: 2.23 sec, Search Time: 0.05 sec

2: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 159.80it/s]


Build Time: 23.86 sec, Search Time: 0.12 sec

3: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 31.22 sec, Search Time: 0.04 sec

19: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 541.09it/s]


Build Time: 3.68 sec, Search Time: 0.04 sec

20: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 631.94it/s]

Build Time: 34.47 sec, Search Time: 0.16 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 2.23 sec, Search: 0.05 sec
Data:  100000 points, Queries:   20 => Build: 23.86 sec, Search: 0.12 sec
Data:  100000 points, Queries:   20 => Build: 26.43 sec, Search: 0.05 sec
Data:   10000 points, Queries:  100 => Build: 4.91 sec, Search: 0.14 sec
Data:  100000 points, Queries:  500 => Build: 26.52 sec, Search: 0.96 sec
Data:  100000 points, Queries:   20 => Build: 32.03 sec, Search: 0.05 sec
Data:   10000 points, Queries:  100 => Build: 2.55 sec, Search: 0.12 sec
Data:   10000 points, Queries:   20 => Build: 3.63 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 25.55 sec, Search: 0.18 sec
Data: 1000000 points, Queries:  500 => Build: 314.75 sec, Search: 1.98 sec
Data:   10000 points, Queries:  100 => Build: 4.37 sec, Search: 0.21 sec
Data:  100000 points, Queries:   20 => Build: 34.72 sec, Search: 0.04 sec
Data:  100000 points, Queries:  100 => B

**------VP MWV Analysis---------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 662.39',
    'Total Time: 671.84',
    'Total Time: 677.20'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 670.4767 sec, Std Dev: 7.4985 sec, CV: 1.12%


----------

**------------------Ball Tree--------------------**

In [ ]:
import numpy as np
from collections import deque

#*-----------------------------------------------------------------------------
class BallNode:
    def __init__(self):
        self.center = None
        self.radius = None
        self.left = None
        self.right = None
        self.points = None
        self.pivot1 = None
        self.pivot2 = None

#*-----------------------------------------------------------------------------
class BallTree:
    def __init__(self, leaf_size=40):
        self.leaf_size = leaf_size
        self.root = None

    #*-----------------------------------------------------------------------------
    def _find_furthest_points(self, points):
        # First find point furthest from origin as first pivot
        distances = np.sum(points ** 2, axis=1)
        p1_idx = np.argmax(distances)
        pivot1 = points[p1_idx]

        # Find point furthest from first pivot
        distances = np.sum((points - pivot1) ** 2, axis=1)
        p2_idx = np.argmax(distances)
        pivot2 = points[p2_idx]

        return pivot1, pivot2

    #*-----------------------------------------------------------------------------
    def _compute_distances(self, points, pivot):
        return np.sqrt(np.sum((points - pivot) ** 2, axis=1))

    #*-----------------------------------------------------------------------------
    def build(self, points):
        if points is None or len(points) == 0:
            return None

        points = np.array(points)
        self.root = self._build_recursive(points)
        return self.root

    #*-----------------------------------------------------------------------------
    def _build_recursive(self, points):
        if len(points) <= self.leaf_size:
            node = BallNode()
            node.points = points
            node.center = np.mean(points, axis=0)
            if len(points) > 1:
                node.radius = np.max(self._compute_distances(points, node.center))
            else:
                node.radius = 0
            return node

        node = BallNode()

        # Find two furthest points as pivots
        pivot1, pivot2 = self._find_furthest_points(points)
        node.pivot1 = pivot1
        node.pivot2 = pivot2

        # Assign points to closest pivot
        dist_to_p1 = self._compute_distances(points, pivot1)
        dist_to_p2 = self._compute_distances(points, pivot2)

        left_mask = dist_to_p1 <= dist_to_p2
        left_points = points[left_mask]
        right_points = points[~left_mask]

        # Compute node properties
        node.center = np.mean(points, axis=0)
        node.radius = np.max(self._compute_distances(points, node.center))

        # Recursively build subtrees
        node.left = self._build_recursive(left_points)
        node.right = self._build_recursive(right_points)

        return node

    #*-----------------------------------------------------------------------------
    def search(self, point):
        if self.root is None:
            return float('inf'), None

        point = np.array(point)
        best_dist = float('inf')
        best_point = None

        # Use a priority queue for best-first search
        # Each entry is (lower_bound_distance, node)
        queue = deque([(0, self.root)])

        while queue:
            _, node = queue.popleft()  # Get the node with lowest lower bound distance

            # Calculate distance from query point to node center
            dist_to_center = np.linalg.norm(point - node.center)

            # If the closest possible point in this ball is farther than our best,
            # we can skip this subtree
            if dist_to_center - node.radius > best_dist:
                continue

            if node.points is not None:
                # Leaf node - check all points
                distances = np.linalg.norm(node.points - point, axis=1)
                min_idx = np.argmin(distances)
                min_dist = distances[min_idx]

                if min_dist < best_dist:
                    best_dist = min_dist
                    best_point = node.points[min_idx]
            else:
                # Internal node - decide which child to visit first
                dist_to_p1 = np.linalg.norm(point - node.pivot1)
                dist_to_p2 = np.linalg.norm(point - node.pivot2)

                # Calculate lower bound distances to both subtrees
                # and visit them in order of potential closeness
                if dist_to_p1 <= dist_to_p2:
                    # Visit left subtree first, then right
                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        queue.appendleft((left_lb, node.left))  # Add to front to visit next

                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        if right_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((right_lb, node.right))
                else:
                    # Visit right subtree first, then left
                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        queue.appendleft((right_lb, node.right))  # Add to front to visit next

                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        if left_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((left_lb, node.left))

        return best_dist, best_point
    #*-----------------------------------------------------------------------------

    def insert(self, point):
        """
        Insert a new point into the Ball Tree.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        # Path to keep track of nodes to update
        path = []

        # Traverse the tree to find the appropriate leaf
        current = self.root
        path.append(current)

        while current.points is None:  # Continue until we reach a leaf node
            # Calculate distances to pivots
            dist_to_p1 = np.sqrt(np.sum((point - current.pivot1) ** 2))
            dist_to_p2 = np.sqrt(np.sum((point - current.pivot2) ** 2))

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid)
        # New centroid = (n * old_centroid + new_point) / (n + 1)
        current.center = (n * current.center + point) / (n + 1)

        # Update the radius if necessary
        dist_to_center = np.sqrt(np.sum((point - current.center) ** 2))
        current.radius = max(current.radius, dist_to_center)

        # Update centers and radii of ancestor nodes
        path.pop()  # Remove the leaf node as we've already updated it
        self._update_ancestors(path, point)

    #*-----------------------------------------------------------------------------
    def _update_ancestors(self, path, point):
        """
        Update the center and radius of all nodes in the path after insertion.

        Parameters:
            path (list): List of nodes from root to leaf (excluding the leaf)
            point (np.ndarray): The point that was inserted
        """
        # Process nodes from bottom to top (leaf's parent first)
        for node in reversed(path):
            # For center, we would ideally recompute based on all points
            # but as an approximation, we'll slightly shift it toward the new point
            # This way we avoid having to recursively gather all points

            # We'll apply a small weight to the new point based on estimated subtree size
            # Assume a balanced tree with approximately 2^depth nodes at the bottom
            depth = len(path) - path.index(node)
            weight = 1.0 / (2**depth)

            # Update center with minimal shift toward new point
            node.center = (1 - weight) * node.center + weight * point

            # Check if radius needs update by calculating distance from new point to center
            dist_to_center = np.sqrt(np.sum((point - node.center) ** 2))

            # Only update radius if the new point is outside the current ball
            if dist_to_center > node.radius:
                node.radius = dist_to_center

    #*-----------------------------------------------------------------------------
    def insert_opt(self, point):
        """
        Insert a new point into the Ball Tree with optimized performance.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        current = self.root
        nodes_to_update = []  # Store nodes that need radius updates

        # Traverse the tree to find the appropriate leaf
        while current.points is None:  # Continue until we reach a leaf node
            # Check if point is within current node's radius
            dist_to_center = np.linalg.norm(point - current.center)
            if dist_to_center > current.radius:
                nodes_to_update.append((current, dist_to_center))

            # Calculate distances to pivots (only once per level)
            dist_to_p1 = np.linalg.norm(point - current.pivot1)
            dist_to_p2 = np.linalg.norm(point - current.pivot2)

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid) - avoid creating temporary arrays
        current.center *= n
        current.center += point
        current.center /= (n + 1)

        # Update the radius if necessary
        dist_to_center = np.linalg.norm(point - current.center)
        if dist_to_center > current.radius:
            current.radius = dist_to_center

        # Update only radii of nodes that need updates
        for node, dist in nodes_to_update:
            node.radius = max(node.radius, dist)
#*-----------------------------------------------------------------------------

-----

**--------Ball Tree (Run 1)----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 650

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 1692.04it/s]

Build Time: 0.02 sec, Search Time: 0.01 sec

2:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===



Querying: 100%|██████████| 20/20 [00:00<00:00, 401.87it/s]


Build Time: 0.26 sec, Search Time: 0.05 sec

3:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 6.98 sec, Search Time: 0.16 sec

19:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 123.57it/s]


Build Time: 6.98 sec, Search Time: 0.16 sec

20:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 123.82it/s]

Build Time: 7.34 sec, Search Time: 0.80 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 0.02 sec, Search: 0.01 sec
Data:  100000 points, Queries:   20 => Build: 0.26 sec, Search: 0.05 sec
Data:  100000 points, Queries:   20 => Build: 0.53 sec, Search: 0.09 sec
Data:   10000 points, Queries:  100 => Build: 0.55 sec, Search: 0.36 sec
Data:  100000 points, Queries:  500 => Build: 0.84 sec, Search: 1.94 sec
Data:  100000 points, Queries:   20 => Build: 1.12 sec, Search: 0.08 sec
Data:   10000 points, Queries:  100 => Build: 1.16 sec, Search: 0.47 sec
Data:   10000 points, Queries:   20 => Build: 1.17 sec, Search: 0.09 sec
Data:  100000 points, Queries:  100 => Build: 1.52 sec, Search: 0.48 sec
Data: 1000000 points, Queries:  500 => Build: 5.23 sec, Search: 3.49 sec
Data:   10000 points, Queries:  100 => Build: 5.06 sec, Search: 0.70 sec
Data:  100000 points, Queries:   20 => Build: 5.38 sec, Search: 0.16 sec
Data:  100000 points, Queries:  100 => Build: 5.6

**-------Ball Run 2----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 650

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 2133.20it/s]

Build Time: 0.01 sec, Search Time: 0.01 sec

2:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===



Querying: 100%|██████████| 20/20 [00:00<00:00, 378.90it/s]


Build Time: 0.25 sec, Search Time: 0.05 sec

3:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 6.90 sec, Search Time: 0.15 sec

19:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 132.70it/s]


Build Time: 6.91 sec, Search Time: 0.15 sec

20:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 140.75it/s]

Build Time: 7.25 sec, Search Time: 0.71 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 0.01 sec, Search: 0.01 sec
Data:  100000 points, Queries:   20 => Build: 0.25 sec, Search: 0.05 sec
Data:  100000 points, Queries:   20 => Build: 0.51 sec, Search: 0.06 sec
Data:   10000 points, Queries:  100 => Build: 0.54 sec, Search: 0.32 sec
Data:  100000 points, Queries:  500 => Build: 0.86 sec, Search: 1.98 sec
Data:  100000 points, Queries:   20 => Build: 1.13 sec, Search: 0.09 sec
Data:   10000 points, Queries:  100 => Build: 1.18 sec, Search: 0.44 sec
Data:   10000 points, Queries:   20 => Build: 1.20 sec, Search: 0.09 sec
Data:  100000 points, Queries:  100 => Build: 1.52 sec, Search: 0.47 sec
Data: 1000000 points, Queries:  500 => Build: 4.96 sec, Search: 3.30 sec
Data:   10000 points, Queries:  100 => Build: 5.01 sec, Search: 0.60 sec
Data:  100000 points, Queries:   20 => Build: 5.36 sec, Search: 0.15 sec
Data:  100000 points, Queries:  100 => Build: 5.6

**---------Ball Tree Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 650

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 1487.45it/s]

Build Time: 0.01 sec, Search Time: 0.01 sec

2:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===



Querying: 100%|██████████| 20/20 [00:00<00:00, 381.06it/s]


Build Time: 0.25 sec, Search Time: 0.05 sec

3:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


...

Build Time: 6.88 sec, Search Time: 0.16 sec

19:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 131.66it/s]


Build Time: 7.05 sec, Search Time: 0.15 sec

20:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 116.39it/s]

Build Time: 7.45 sec, Search Time: 0.85 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:   20 => Build: 0.01 sec, Search: 0.01 sec
Data:  100000 points, Queries:   20 => Build: 0.25 sec, Search: 0.05 sec
Data:  100000 points, Queries:   20 => Build: 0.52 sec, Search: 0.05 sec
Data:   10000 points, Queries:  100 => Build: 0.59 sec, Search: 0.29 sec
Data:  100000 points, Queries:  500 => Build: 0.85 sec, Search: 1.87 sec
Data:  100000 points, Queries:   20 => Build: 1.13 sec, Search: 0.07 sec
Data:   10000 points, Queries:  100 => Build: 1.16 sec, Search: 0.48 sec
Data:   10000 points, Queries:   20 => Build: 1.20 sec, Search: 0.10 sec
Data:  100000 points, Queries:  100 => Build: 1.53 sec, Search: 0.49 sec
Data: 1000000 points, Queries:  500 => Build: 5.00 sec, Search: 3.62 sec
Data:   10000 points, Queries:  100 => Build: 5.04 sec, Search: 0.73 sec
Data:  100000 points, Queries:   20 => Build: 5.43 sec, Search: 0.13 sec
Data:  100000 points, Queries:  100 => Build: 5.8

**----------Ball Tree Analysis---------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 90.56',
    'Total Time: 89.33'
    'Total Time: 90.80'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 89.9450 sec, Std Dev: 0.8697 sec, CV: 0.97%


---------------

**----------QuickSelect Algorithm-----------**

In [10]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

---------------------------------------

------------------------------------------------

**--------KD Hybrid---------**

In [11]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

-------------------------

**----------KD Hybrid Sqrt Run 1-------------**

In [12]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0337 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2732.80it/s]

Total Search Time: 0.019743 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.7009 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 699.59it/s]


...

Total Search Time: 0.926114 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87359.60it/s]


Insert Time: 1.1473 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 82.03it/s]

Total Search Time: 1.988066 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0337 sec, Total Search Time: 0.019743 sec, Total Time: 0.0535 sec
Points: 110,000, Build Time: 0.7009 sec, Total Search Time: 0.058425 sec, Total Time: 0.7593 sec
Points: 210,000, Insert Time: 0.8219 sec, Total Search Time: 0.124797 sec, Total Time: 0.9467 sec
Points: 220,000, Insert Time: 0.1189 sec, Total Search Time: 0.273357 sec, Total Time: 0.3922 sec
Points: 320,000, Build Time: 2.2717 sec, Total Search Time: 1.564485 sec, Total Time: 3.8362 sec
Points: 420,000, Insert Time: 1.0368 sec, Total Search Time: 0.196759 sec, Total Time: 1.2336 sec
Points: 430,000, Insert Time: 0.0918 sec, Total Search Time: 0.473891 sec, Total Time: 0.5657 sec
Points: 440,000, Insert Time: 0.0855 sec, Total Search Time: 0.218863 sec, Total Time: 0.3044 sec
Points: 540,000, Insert Time: 1.0341 sec, Total Search Time: 0.565483 sec, Total Time: 1.5996 sec
Points: 1,540,000, Build Time: 15.0728 sec, Total

**----------KD Hybrid Sqrt Run 2-----------**

In [17]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0239 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2835.04it/s]

Total Search Time: 0.012441 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.2735 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 771.74it/s]


...

Total Search Time: 0.860729 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87844.57it/s]


Insert Time: 1.1416 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 81.78it/s]

Total Search Time: 2.008808 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0239 sec, Total Search Time: 0.012441 sec, Total Time: 0.0364 sec
Points: 110,000, Build Time: 1.2735 sec, Total Search Time: 0.059978 sec, Total Time: 1.3335 sec
Points: 210,000, Insert Time: 0.8824 sec, Total Search Time: 0.165658 sec, Total Time: 1.0481 sec
Points: 220,000, Insert Time: 0.1009 sec, Total Search Time: 0.336921 sec, Total Time: 0.4378 sec
Points: 320,000, Build Time: 2.4844 sec, Total Search Time: 1.628322 sec, Total Time: 4.1127 sec
Points: 420,000, Insert Time: 0.9860 sec, Total Search Time: 0.208970 sec, Total Time: 1.1949 sec
Points: 430,000, Insert Time: 0.1014 sec, Total Search Time: 0.460200 sec, Total Time: 0.5616 sec
Points: 440,000, Insert Time: 0.0938 sec, Total Search Time: 0.245424 sec, Total Time: 0.3392 sec
Points: 540,000, Insert Time: 0.9515 sec, Total Search Time: 0.585426 sec, Total Time: 1.5369 sec
Points: 1,540,000, Build Time: 13.5747 sec, Total

**-----------KD Hybrid Sqrt Run 3---------**

In [22]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0257 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2437.77it/s]

Total Search Time: 0.014035 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.0509 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 745.67it/s]


...

Total Search Time: 1.052187 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 82667.77it/s]


Insert Time: 1.2119 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 82.19it/s]

Total Search Time: 1.917747 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0257 sec, Total Search Time: 0.014035 sec, Total Time: 0.0397 sec
Points: 110,000, Build Time: 1.0509 sec, Total Search Time: 0.066493 sec, Total Time: 1.1174 sec
Points: 210,000, Insert Time: 0.7983 sec, Total Search Time: 0.140980 sec, Total Time: 0.9393 sec
Points: 220,000, Insert Time: 0.0868 sec, Total Search Time: 0.298571 sec, Total Time: 0.3853 sec
Points: 320,000, Build Time: 2.6945 sec, Total Search Time: 1.606228 sec, Total Time: 4.3008 sec
Points: 420,000, Insert Time: 0.9518 sec, Total Search Time: 0.234275 sec, Total Time: 1.1861 sec
Points: 430,000, Insert Time: 0.0924 sec, Total Search Time: 0.534827 sec, Total Time: 0.6273 sec
Points: 440,000, Insert Time: 0.1219 sec, Total Search Time: 0.270134 sec, Total Time: 0.3921 sec
Points: 540,000, Insert Time: 1.0566 sec, Total Search Time: 0.656422 sec, Total Time: 1.7130 sec
Points: 1,540,000, Build Time: 14.1078 sec, Total

**-----------KD Hybrid Sqrt Analysis-----------**

In [27]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time:: 56.9950',
    'Total Time:: 56.8248',
    'Total Time:: 57.9087'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 57.2428 sec, Std Dev: 0.5829 sec, CV: 1.02%


---------------------------------------

**------KD Hybrid Log Run 1--------**

In [13]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0237 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2796.67it/s]


Total Search Time: 0.012814 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.6707 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 672.98it/s]


Total Search Time: 0.060181 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.825654 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 95623.83it/s]


Insert Time: 1.0495 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 75.39it/s]

Total Search Time: 2.148688 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0237 sec, Total Search Time: 0.012814 sec, Total Time: 0.0365 sec
Points: 110,000, Build Time: 0.6707 sec, Total Search Time: 0.060181 sec, Total Time: 0.7308 sec
Points: 210,000, Insert Time: 0.9124 sec, Total Search Time: 0.145604 sec, Total Time: 1.0580 sec
Points: 220,000, Insert Time: 0.1058 sec, Total Search Time: 0.300285 sec, Total Time: 0.4061 sec
Points: 320,000, Insert Time: 0.9129 sec, Total Search Time: 1.561673 sec, Total Time: 2.4746 sec
Points: 420,000, Insert Time: 0.8373 sec, Total Search Time: 0.292433 sec, Total Time: 1.1297 sec
Points: 430,000, Insert Time: 0.0765 sec, Total Search Time: 0.508591 sec, Total Time: 0.5851 sec
Points: 440,000, Insert Time: 0.0765 sec, Total Search Time: 0.411131 sec, Total Time: 0.4876 sec
Points: 540,000, Insert Time: 0.7934 sec, Total Search Time: 0.665334 sec, Total Time: 1.4588 sec
Points: 1,540,000, Build Time: 15.4546 sec, Tota

**-------KD Hybrid Log Run 2-----------**

In [18]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0291 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2569.65it/s]


Total Search Time: 0.014998 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.0811 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 797.24it/s]


Total Search Time: 0.061716 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 1.040765 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 90527.50it/s]


Insert Time: 1.1094 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 83.97it/s]

Total Search Time: 2.055176 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0291 sec, Total Search Time: 0.014998 sec, Total Time: 0.0441 sec
Points: 110,000, Build Time: 1.0811 sec, Total Search Time: 0.061716 sec, Total Time: 1.1428 sec
Points: 210,000, Insert Time: 0.7926 sec, Total Search Time: 0.164295 sec, Total Time: 0.9569 sec
Points: 220,000, Insert Time: 0.0887 sec, Total Search Time: 0.314261 sec, Total Time: 0.4030 sec
Points: 320,000, Insert Time: 0.8344 sec, Total Search Time: 1.506836 sec, Total Time: 2.3413 sec
Points: 420,000, Insert Time: 0.8020 sec, Total Search Time: 0.297826 sec, Total Time: 1.0999 sec
Points: 430,000, Insert Time: 0.0751 sec, Total Search Time: 0.567618 sec, Total Time: 0.6427 sec
Points: 440,000, Insert Time: 0.0878 sec, Total Search Time: 0.372616 sec, Total Time: 0.4605 sec
Points: 540,000, Insert Time: 0.7666 sec, Total Search Time: 0.823490 sec, Total Time: 1.5901 sec
Points: 1,540,000, Build Time: 13.7492 sec, Tota

**------KD Hybrid Log Run 3------**

In [23]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0260 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2885.86it/s]


Total Search Time: 0.013318 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.5211 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 740.96it/s]


Total Search Time: 0.066187 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.895148 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 88418.20it/s]


Insert Time: 1.1337 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 78.82it/s]

Total Search Time: 2.064211 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0260 sec, Total Search Time: 0.013318 sec, Total Time: 0.0393 sec
Points: 110,000, Build Time: 1.5211 sec, Total Search Time: 0.066187 sec, Total Time: 1.5873 sec
Points: 210,000, Insert Time: 0.8363 sec, Total Search Time: 0.121773 sec, Total Time: 0.9580 sec
Points: 220,000, Insert Time: 0.0851 sec, Total Search Time: 0.287921 sec, Total Time: 0.3730 sec
Points: 320,000, Insert Time: 0.8380 sec, Total Search Time: 1.450967 sec, Total Time: 2.2889 sec
Points: 420,000, Insert Time: 0.8307 sec, Total Search Time: 0.237960 sec, Total Time: 1.0686 sec
Points: 430,000, Insert Time: 0.0776 sec, Total Search Time: 0.554975 sec, Total Time: 0.6325 sec
Points: 440,000, Insert Time: 0.0842 sec, Total Search Time: 0.421525 sec, Total Time: 0.5057 sec
Points: 540,000, Insert Time: 0.8181 sec, Total Search Time: 0.850027 sec, Total Time: 1.6682 sec
Points: 1,540,000, Build Time: 13.9957 sec, Tota

**----------KD Hybrid Log Analysis----------**

In [26]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 56.4452',
    'Total Time:: 55.5101',
    'Total Time:: 55.6081'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 55.8545 sec, Std Dev: 0.5139 sec, CV: 0.92%


--------------------------------

**--------KD Hybrid Log Ratio Run 1---------**

In [14]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.2472 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2451.66it/s]

Total Search Time: 0.017638 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.5551 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 687.35it/s]


...

Total Search Time: 0.846216 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87427.16it/s]


Insert Time: 1.1456 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 81.34it/s]

Total Search Time: 1.930549 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.2472 sec, Total Search Time: 0.017638 sec, Total Time: 0.2648 sec
Points: 110,000, Build Time: 0.5551 sec, Total Search Time: 0.061347 sec, Total Time: 0.6164 sec
Points: 210,000, Insert Time: 0.8302 sec, Total Search Time: 0.122639 sec, Total Time: 0.9528 sec
Points: 220,000, Insert Time: 0.0799 sec, Total Search Time: 0.282902 sec, Total Time: 0.3628 sec
Points: 320,000, Insert Time: 0.7858 sec, Total Search Time: 1.397471 sec, Total Time: 2.1832 sec
Points: 420,000, Insert Time: 0.8735 sec, Total Search Time: 0.284144 sec, Total Time: 1.1576 sec
Points: 430,000, Insert Time: 0.0817 sec, Total Search Time: 0.494075 sec, Total Time: 0.5758 sec
Points: 440,000, Insert Time: 0.0818 sec, Total Search Time: 0.325805 sec, Total Time: 0.4076 sec
Points: 540,000, Insert Time: 0.7675 sec, Total Search Time: 0.863666 sec, Total Time: 1.6311 sec
Points: 1,540,000, Build Time: 15.7719 sec, Tota

**------KD Hybrid Log Ratio Run 2---------**

In [19]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0255 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2854.92it/s]


Total Search Time: 0.013899 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.0280 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 764.57it/s]


Total Search Time: 0.061760 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.876901 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 88521.34it/s]


Insert Time: 1.1338 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 81.11it/s]

Total Search Time: 1.971279 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0255 sec, Total Search Time: 0.013899 sec, Total Time: 0.0394 sec
Points: 110,000, Build Time: 1.0280 sec, Total Search Time: 0.061760 sec, Total Time: 1.0898 sec
Points: 210,000, Insert Time: 0.8652 sec, Total Search Time: 0.142694 sec, Total Time: 1.0079 sec
Points: 220,000, Insert Time: 0.0892 sec, Total Search Time: 0.307142 sec, Total Time: 0.3964 sec
Points: 320,000, Insert Time: 0.8162 sec, Total Search Time: 1.491320 sec, Total Time: 2.3075 sec
Points: 420,000, Insert Time: 0.8321 sec, Total Search Time: 0.364415 sec, Total Time: 1.1966 sec
Points: 430,000, Insert Time: 0.0763 sec, Total Search Time: 0.628672 sec, Total Time: 0.7050 sec
Points: 440,000, Insert Time: 0.0826 sec, Total Search Time: 0.349495 sec, Total Time: 0.4321 sec
Points: 540,000, Insert Time: 0.8547 sec, Total Search Time: 0.680862 sec, Total Time: 1.5355 sec
Points: 1,540,000, Build Time: 13.6323 sec, Tota

**------KD Hybrid Log Ratio Run 3-----**

In [24]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0264 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2722.42it/s]


Total Search Time: 0.012829 sec

Batch 2: Adding 100,000 points
Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.0593 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 757.87it/s]


Total Search Time: 0.061937 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.855206 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 99274.67it/s]


Insert Time: 1.0107 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 77.85it/s]

Total Search Time: 2.114985 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0264 sec, Total Search Time: 0.012829 sec, Total Time: 0.0393 sec
Points: 110,000, Build Time: 1.0593 sec, Total Search Time: 0.061937 sec, Total Time: 1.1213 sec
Points: 210,000, Insert Time: 0.9058 sec, Total Search Time: 0.146692 sec, Total Time: 1.0525 sec
Points: 220,000, Insert Time: 0.1154 sec, Total Search Time: 0.299853 sec, Total Time: 0.4153 sec
Points: 320,000, Insert Time: 0.8186 sec, Total Search Time: 1.461076 sec, Total Time: 2.2797 sec
Points: 420,000, Insert Time: 0.8012 sec, Total Search Time: 0.427367 sec, Total Time: 1.2286 sec
Points: 430,000, Insert Time: 0.0763 sec, Total Search Time: 0.650623 sec, Total Time: 0.7269 sec
Points: 440,000, Insert Time: 0.0808 sec, Total Search Time: 0.415189 sec, Total Time: 0.4960 sec
Points: 540,000, Insert Time: 0.7595 sec, Total Search Time: 0.702012 sec, Total Time: 1.4615 sec
Points: 1,540,000, Build Time: 13.5690 sec, Tota

**-----------KD Hybrid Log Ratio Analysis----------**

In [25]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 56.5710',
    'Total Time:: 54.6638',
    'Total Time:: 55.3865'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 55.5404 sec, Std Dev: 0.9629 sec, CV: 1.73%


-------------------------------------

---------------------------------------

**----------KD Dynamic---------**

In [ ]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous data storage
        if n == 2*self.leaf_size:
            self.leaf_size = n
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

----------------------

**--------KD Dyn Sqrt Run 1-------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0419 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3254.30it/s]

Total Search Time: 0.016009 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.6813 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 813.10it/s]


...

Total Search Time: 0.812409 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 81912.40it/s]


Insert Time: 1.2230 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 96.25it/s]

Total Search Time: 1.683410 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0419 sec, Total Search Time: 0.016009 sec, Total Time: 0.0579 sec
Points: 110,000, Build Time: 0.6813 sec, Total Search Time: 0.055798 sec, Total Time: 0.7371 sec
Points: 210,000, Insert Time: 0.8256 sec, Total Search Time: 0.124919 sec, Total Time: 0.9506 sec
Points: 220,000, Insert Time: 0.0809 sec, Total Search Time: 0.286067 sec, Total Time: 0.3670 sec
Points: 320,000, Build Time: 2.4106 sec, Total Search Time: 1.367547 sec, Total Time: 3.7781 sec
Points: 420,000, Insert Time: 1.0407 sec, Total Search Time: 0.177346 sec, Total Time: 1.2181 sec
Points: 430,000, Insert Time: 0.1367 sec, Total Search Time: 0.410356 sec, Total Time: 0.5470 sec
Points: 440,000, Insert Time: 0.1261 sec, Total Search Time: 0.192395 sec, Total Time: 0.3185 sec
Points: 540,000, Insert Time: 0.9304 sec, Total Search Time: 0.478950 sec, Total Time: 1.4094 sec
Points: 1,540,000, Build Time: 13.8471 sec, Total

**----------KD Dyn Sqrt Run 2--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0344 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3278.46it/s]

Total Search Time: 0.011506 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.5274 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 829.36it/s]


...

Total Search Time: 0.828298 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 79477.92it/s]


Insert Time: 1.2610 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 97.63it/s]

Total Search Time: 1.675563 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0344 sec, Total Search Time: 0.011506 sec, Total Time: 0.0459 sec
Points: 110,000, Build Time: 0.5274 sec, Total Search Time: 0.055408 sec, Total Time: 0.5828 sec
Points: 210,000, Insert Time: 0.9085 sec, Total Search Time: 0.112601 sec, Total Time: 1.0211 sec
Points: 220,000, Insert Time: 0.0942 sec, Total Search Time: 0.276927 sec, Total Time: 0.3711 sec
Points: 320,000, Build Time: 2.7604 sec, Total Search Time: 1.345490 sec, Total Time: 4.1059 sec
Points: 420,000, Insert Time: 0.9883 sec, Total Search Time: 0.199269 sec, Total Time: 1.1876 sec
Points: 430,000, Insert Time: 0.1098 sec, Total Search Time: 0.433469 sec, Total Time: 0.5433 sec
Points: 440,000, Insert Time: 0.0912 sec, Total Search Time: 0.209524 sec, Total Time: 0.3007 sec
Points: 540,000, Insert Time: 1.0083 sec, Total Search Time: 0.513524 sec, Total Time: 1.5218 sec
Points: 1,540,000, Build Time: 14.1360 sec, Total

**----------KD Dyn Sqrt Run 3-----------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0316 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2978.27it/s]

Total Search Time: 0.011584 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.5086 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 800.41it/s]


...

Total Search Time: 0.792880 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 90909.20it/s]


Insert Time: 1.1021 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 100.32it/s]

Total Search Time: 1.639586 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0316 sec, Total Search Time: 0.011584 sec, Total Time: 0.0432 sec
Points: 110,000, Build Time: 0.5086 sec, Total Search Time: 0.057805 sec, Total Time: 0.5664 sec
Points: 210,000, Insert Time: 0.8792 sec, Total Search Time: 0.108807 sec, Total Time: 0.9880 sec
Points: 220,000, Insert Time: 0.0847 sec, Total Search Time: 0.277517 sec, Total Time: 0.3622 sec
Points: 320,000, Build Time: 2.7566 sec, Total Search Time: 1.349130 sec, Total Time: 4.1057 sec
Points: 420,000, Insert Time: 1.0616 sec, Total Search Time: 0.196132 sec, Total Time: 1.2577 sec
Points: 430,000, Insert Time: 0.0918 sec, Total Search Time: 0.440905 sec, Total Time: 0.5327 sec
Points: 440,000, Insert Time: 0.0982 sec, Total Search Time: 0.203330 sec, Total Time: 0.3015 sec
Points: 540,000, Insert Time: 1.0034 sec, Total Search Time: 0.491261 sec, Total Time: 1.4947 sec
Points: 1,540,000, Build Time: 13.9135 sec, Total

**---------KD Dyn Sqrt Analysis--------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 52.1135',
    'Total Time:: 52.7910',
    'Total Time:: 51.8392'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 52.2479 sec, Std Dev: 0.4899 sec, CV: 0.94%


--------------------------

**-------KD Dyn Log Run 1-----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0329 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3532.19it/s]

Total Search Time: 0.010519 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.0221 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 820.32it/s]


...

Total Search Time: 0.810536 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 85127.70it/s]


Insert Time: 1.1769 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 96.90it/s]

Total Search Time: 1.685838 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0329 sec, Total Search Time: 0.010519 sec, Total Time: 0.0434 sec
Points: 110,000, Build Time: 1.0221 sec, Total Search Time: 0.056152 sec, Total Time: 1.0783 sec
Points: 210,000, Insert Time: 0.8773 sec, Total Search Time: 0.124095 sec, Total Time: 1.0014 sec
Points: 220,000, Insert Time: 0.0934 sec, Total Search Time: 0.299916 sec, Total Time: 0.3933 sec
Points: 320,000, Insert Time: 0.8705 sec, Total Search Time: 1.464795 sec, Total Time: 2.3353 sec
Points: 420,000, Insert Time: 0.9242 sec, Total Search Time: 0.247742 sec, Total Time: 1.1720 sec
Points: 430,000, Insert Time: 0.0787 sec, Total Search Time: 0.540080 sec, Total Time: 0.6188 sec
Points: 440,000, Insert Time: 0.0825 sec, Total Search Time: 0.273272 sec, Total Time: 0.3558 sec
Points: 540,000, Insert Time: 0.8014 sec, Total Search Time: 0.672847 sec, Total Time: 1.4743 sec
Points: 1,540,000, Build Time: 13.7239 sec, Tota

**-----KD Dyn Log Run 2--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0353 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3161.22it/s]

Total Search Time: 0.011878 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.1078 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 869.83it/s]


...

Total Search Time: 0.898792 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 84472.52it/s]


Insert Time: 1.1863 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 95.58it/s]

Total Search Time: 1.763276 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0353 sec, Total Search Time: 0.011878 sec, Total Time: 0.0472 sec
Points: 110,000, Build Time: 1.1078 sec, Total Search Time: 0.059085 sec, Total Time: 1.1669 sec
Points: 210,000, Insert Time: 0.8761 sec, Total Search Time: 0.121977 sec, Total Time: 0.9981 sec
Points: 220,000, Insert Time: 0.0941 sec, Total Search Time: 0.296972 sec, Total Time: 0.3910 sec
Points: 320,000, Insert Time: 0.8603 sec, Total Search Time: 1.472097 sec, Total Time: 2.3324 sec
Points: 420,000, Insert Time: 0.8883 sec, Total Search Time: 0.256541 sec, Total Time: 1.1448 sec
Points: 430,000, Insert Time: 0.0782 sec, Total Search Time: 0.520936 sec, Total Time: 0.5992 sec
Points: 440,000, Insert Time: 0.0946 sec, Total Search Time: 0.255897 sec, Total Time: 0.3505 sec
Points: 540,000, Insert Time: 0.8669 sec, Total Search Time: 0.645563 sec, Total Time: 1.5124 sec
Points: 1,540,000, Build Time: 14.0854 sec, Tota

**-----KD Dyn Log Run 3------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0414 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3374.07it/s]

Total Search Time: 0.010991 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.0862 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 777.44it/s]


...

Total Search Time: 0.791579 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87029.16it/s]


Insert Time: 1.1516 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 98.21it/s]

Total Search Time: 1.744208 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0414 sec, Total Search Time: 0.010991 sec, Total Time: 0.0524 sec
Points: 110,000, Build Time: 1.0862 sec, Total Search Time: 0.061468 sec, Total Time: 1.1477 sec
Points: 210,000, Insert Time: 1.0086 sec, Total Search Time: 0.120397 sec, Total Time: 1.1290 sec
Points: 220,000, Insert Time: 0.0934 sec, Total Search Time: 0.272985 sec, Total Time: 0.3664 sec
Points: 320,000, Insert Time: 0.9515 sec, Total Search Time: 1.465366 sec, Total Time: 2.4168 sec
Points: 420,000, Insert Time: 0.9438 sec, Total Search Time: 0.244823 sec, Total Time: 1.1886 sec
Points: 430,000, Insert Time: 0.0898 sec, Total Search Time: 0.545503 sec, Total Time: 0.6353 sec
Points: 440,000, Insert Time: 0.0957 sec, Total Search Time: 0.270795 sec, Total Time: 0.3665 sec
Points: 540,000, Insert Time: 0.9011 sec, Total Search Time: 0.686294 sec, Total Time: 1.5874 sec
Points: 1,540,000, Build Time: 14.1877 sec, Tota

**------KD Dyn Log Analysis----------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 50.7252',
    'Total Time:: 51.8719',
    'Total Time:: 51.8462'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 51.4811 sec, Std Dev: 0.6548 sec, CV: 1.27%


------------------

**------KD Tree Dynamic Log Ratio Run 1--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0317 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3475.85it/s]

Total Search Time: 0.010588 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.0796 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 796.26it/s]


...

Total Search Time: 0.818630 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87901.01it/s]


Insert Time: 1.1409 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 95.20it/s]

Total Search Time: 1.734331 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0317 sec, Total Search Time: 0.010588 sec, Total Time: 0.0423 sec
Points: 110,000, Build Time: 1.0796 sec, Total Search Time: 0.058692 sec, Total Time: 1.1383 sec
Points: 210,000, Insert Time: 0.8564 sec, Total Search Time: 0.116165 sec, Total Time: 0.9725 sec
Points: 220,000, Insert Time: 0.0935 sec, Total Search Time: 0.281546 sec, Total Time: 0.3750 sec
Points: 320,000, Insert Time: 0.8858 sec, Total Search Time: 1.429169 sec, Total Time: 2.3150 sec
Points: 420,000, Insert Time: 0.9863 sec, Total Search Time: 0.238036 sec, Total Time: 1.2243 sec
Points: 430,000, Insert Time: 0.0855 sec, Total Search Time: 0.520751 sec, Total Time: 0.6062 sec
Points: 440,000, Insert Time: 0.0994 sec, Total Search Time: 0.265606 sec, Total Time: 0.3650 sec
Points: 540,000, Insert Time: 1.1054 sec, Total Search Time: 0.649783 sec, Total Time: 1.7552 sec
Points: 1,540,000, Build Time: 13.9481 sec, Tota

**--------KD Dyn Log Ratio Run 2------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0336 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3427.42it/s]

Total Search Time: 0.010598 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 1.0652 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 851.18it/s]


...

Total Search Time: 0.892067 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87254.65it/s]


Insert Time: 1.1478 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 99.45it/s]

Total Search Time: 1.665270 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0336 sec, Total Search Time: 0.010598 sec, Total Time: 0.0442 sec
Points: 110,000, Build Time: 1.0652 sec, Total Search Time: 0.056638 sec, Total Time: 1.1218 sec
Points: 210,000, Insert Time: 0.8566 sec, Total Search Time: 0.115533 sec, Total Time: 0.9721 sec
Points: 220,000, Insert Time: 0.0916 sec, Total Search Time: 0.282614 sec, Total Time: 0.3742 sec
Points: 320,000, Insert Time: 0.8357 sec, Total Search Time: 1.445769 sec, Total Time: 2.2815 sec
Points: 420,000, Insert Time: 0.9482 sec, Total Search Time: 0.281243 sec, Total Time: 1.2295 sec
Points: 430,000, Insert Time: 0.0928 sec, Total Search Time: 0.541244 sec, Total Time: 0.6340 sec
Points: 440,000, Insert Time: 0.0924 sec, Total Search Time: 0.270218 sec, Total Time: 0.3626 sec
Points: 540,000, Insert Time: 0.9879 sec, Total Search Time: 0.695133 sec, Total Time: 1.6831 sec
Points: 1,540,000, Build Time: 14.7296 sec, Tota

**-------KD Dynamic Log Ratio Run 3----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0327 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3254.17it/s]

Total Search Time: 0.012469 sec

Batch 2: Adding 100,000 points


Rebuilding KD Tree from scratch with 110,000 points.
Build Time: 0.5669 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 781.96it/s]


...

Total Search Time: 0.854421 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 90681.13it/s]


Insert Time: 1.1047 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 99.53it/s]

Total Search Time: 1.666654 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0327 sec, Total Search Time: 0.012469 sec, Total Time: 0.0452 sec
Points: 110,000, Build Time: 0.5669 sec, Total Search Time: 0.056483 sec, Total Time: 0.6234 sec
Points: 210,000, Insert Time: 0.9090 sec, Total Search Time: 0.119672 sec, Total Time: 1.0287 sec
Points: 220,000, Insert Time: 0.0907 sec, Total Search Time: 0.273540 sec, Total Time: 0.3643 sec
Points: 320,000, Insert Time: 0.9252 sec, Total Search Time: 1.509699 sec, Total Time: 2.4349 sec
Points: 420,000, Insert Time: 0.9357 sec, Total Search Time: 0.252562 sec, Total Time: 1.1883 sec
Points: 430,000, Insert Time: 0.0925 sec, Total Search Time: 0.562539 sec, Total Time: 0.6550 sec
Points: 440,000, Insert Time: 0.1208 sec, Total Search Time: 0.287993 sec, Total Time: 0.4087 sec
Points: 540,000, Insert Time: 0.8495 sec, Total Search Time: 0.678220 sec, Total Time: 1.5278 sec
Points: 1,540,000, Build Time: 15.1739 sec, Tota

**-------KD Log Ratio Analysis--------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 51.5583',
    'Total Time:: 52.8566',
    'Total Time:: 52.5301'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 52.3150 sec, Std Dev: 0.6753 sec, CV: 1.29%


---------------------

----------------------

**----------VP Tree Hybrid------**

In [ ]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon
        self._prepared = False  # Track if the tree is prepared for search

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """
        if self._prepared:
            return  # Only convert once unless new insertions occurred

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

        self._prepared = True

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search
        if not self._prepared:
            self.prepare_for_search()

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        # Mark tree as needing preparation before next search
        self._prepared = False

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

----------------

**-------VP Hybrid Sqrt Run 1------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0121 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 12178.58it/s]


Total Search Time: 0.003277 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1007 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 10123.83it/s]


Total Search Time: 0.004396 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.617181 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 119429.29it/s]


Insert Time: 0.8394 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 33.47it/s]

Total Search Time: 3.095745 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0121 sec, Total Search Time: 0.003277 sec, Total Time: 0.0153 sec
Points: 110,000, Build Time: 0.1007 sec, Total Search Time: 0.004396 sec, Total Time: 0.1051 sec
Points: 210,000, Insert Time: 0.8395 sec, Total Search Time: 0.100776 sec, Total Time: 0.9403 sec
Points: 220,000, Insert Time: 0.1125 sec, Total Search Time: 0.446949 sec, Total Time: 0.5594 sec
Points: 320,000, Build Time: 0.3736 sec, Total Search Time: 0.036584 sec, Total Time: 0.4102 sec
Points: 420,000, Insert Time: 0.8359 sec, Total Search Time: 0.094372 sec, Total Time: 0.9303 sec
Points: 430,000, Insert Time: 0.0867 sec, Total Search Time: 0.431960 sec, Total Time: 0.5186 sec
Points: 440,000, Insert Time: 0.0957 sec, Total Search Time: 0.111245 sec, Total Time: 0.2069 sec
Points: 540,000, Insert Time: 0.8638 sec, Total Search Time: 0.879624 sec, Total Time: 1.7435 sec
Points: 1,540,000, Build Time: 2.3814 sec, Total 

**------VP Hybrid Sqrt Run 2-----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0428 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 11060.93it/s]


Total Search Time: 0.003916 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1087 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 7560.03it/s]


Total Search Time: 0.005007 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.517198 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 90528.97it/s]


Insert Time: 1.1068 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 39.59it/s]

Total Search Time: 2.631136 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0428 sec, Total Search Time: 0.003916 sec, Total Time: 0.0467 sec
Points: 110,000, Build Time: 0.1087 sec, Total Search Time: 0.005007 sec, Total Time: 0.1137 sec
Points: 210,000, Insert Time: 0.8327 sec, Total Search Time: 0.094387 sec, Total Time: 0.9271 sec
Points: 220,000, Insert Time: 0.0945 sec, Total Search Time: 0.426686 sec, Total Time: 0.5212 sec
Points: 320,000, Build Time: 0.5006 sec, Total Search Time: 0.064468 sec, Total Time: 0.5650 sec
Points: 420,000, Insert Time: 0.8268 sec, Total Search Time: 0.097543 sec, Total Time: 0.9244 sec
Points: 430,000, Insert Time: 0.0843 sec, Total Search Time: 0.421298 sec, Total Time: 0.5056 sec
Points: 440,000, Insert Time: 0.0842 sec, Total Search Time: 0.109681 sec, Total Time: 0.1939 sec
Points: 540,000, Insert Time: 0.8231 sec, Total Search Time: 0.886454 sec, Total Time: 1.7095 sec
Points: 1,540,000, Build Time: 2.3885 sec, Total 

**-------VP Hybrid Sqrt Run 3------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0167 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 14852.35it/s]


Total Search Time: 0.003187 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1046 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 9392.69it/s]


Total Search Time: 0.005093 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.514687 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 118647.93it/s]


Insert Time: 0.8458 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 33.30it/s]

Total Search Time: 3.108363 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0167 sec, Total Search Time: 0.003187 sec, Total Time: 0.0199 sec
Points: 110,000, Build Time: 0.1046 sec, Total Search Time: 0.005093 sec, Total Time: 0.1097 sec
Points: 210,000, Insert Time: 0.8470 sec, Total Search Time: 0.092516 sec, Total Time: 0.9395 sec
Points: 220,000, Insert Time: 0.0845 sec, Total Search Time: 0.433078 sec, Total Time: 0.5176 sec
Points: 320,000, Build Time: 0.4858 sec, Total Search Time: 0.036696 sec, Total Time: 0.5225 sec
Points: 420,000, Insert Time: 0.8191 sec, Total Search Time: 0.090162 sec, Total Time: 0.9093 sec
Points: 430,000, Insert Time: 0.0856 sec, Total Search Time: 0.434311 sec, Total Time: 0.5199 sec
Points: 440,000, Insert Time: 0.0911 sec, Total Search Time: 0.112148 sec, Total Time: 0.2033 sec
Points: 540,000, Insert Time: 0.8531 sec, Total Search Time: 0.887387 sec, Total Time: 1.7405 sec
Points: 1,540,000, Build Time: 2.2674 sec, Total 

**-----VP Hybrid Sqrt Analysis--------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 30.3600',
    'Total Time:: 31.2782',
    'Total Time:: 30.0873'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 30.5752 sec, Std Dev: 0.6239 sec, CV: 2.04%


-------------

**-----VP Hybrid Log Run 1------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0177 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 17331.83it/s]


Total Search Time: 0.003090 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1343 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 11297.79it/s]


Total Search Time: 0.003937 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.513182 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 120646.80it/s]


Insert Time: 0.8310 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 33.57it/s]

Total Search Time: 3.088154 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0177 sec, Total Search Time: 0.003090 sec, Total Time: 0.0207 sec
Points: 110,000, Build Time: 0.1343 sec, Total Search Time: 0.003937 sec, Total Time: 0.1382 sec
Points: 210,000, Insert Time: 0.8572 sec, Total Search Time: 0.090959 sec, Total Time: 0.9482 sec
Points: 220,000, Insert Time: 0.0848 sec, Total Search Time: 0.438119 sec, Total Time: 0.5229 sec
Points: 320,000, Insert Time: 0.8510 sec, Total Search Time: 4.113404 sec, Total Time: 4.9644 sec
Points: 420,000, Insert Time: 0.8438 sec, Total Search Time: 0.293335 sec, Total Time: 1.1371 sec
Points: 430,000, Insert Time: 0.1295 sec, Total Search Time: 1.317030 sec, Total Time: 1.4465 sec
Points: 440,000, Insert Time: 0.0838 sec, Total Search Time: 0.308530 sec, Total Time: 0.3923 sec
Points: 540,000, Insert Time: 0.8217 sec, Total Search Time: 1.745198 sec, Total Time: 2.5669 sec
Points: 1,540,000, Build Time: 2.3622 sec, Total

**-------VP Hybrid Log Run 2-----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0193 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 10543.75it/s]


Total Search Time: 0.004111 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1071 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 9188.97it/s]


Total Search Time: 0.004806 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.514468 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87971.66it/s]


Insert Time: 1.1393 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 39.82it/s]

Total Search Time: 2.618093 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0193 sec, Total Search Time: 0.004111 sec, Total Time: 0.0234 sec
Points: 110,000, Build Time: 0.1071 sec, Total Search Time: 0.004806 sec, Total Time: 0.1119 sec
Points: 210,000, Insert Time: 0.8616 sec, Total Search Time: 0.093838 sec, Total Time: 0.9554 sec
Points: 220,000, Insert Time: 0.0854 sec, Total Search Time: 0.444219 sec, Total Time: 0.5296 sec
Points: 320,000, Insert Time: 0.8664 sec, Total Search Time: 4.153392 sec, Total Time: 5.0198 sec
Points: 420,000, Insert Time: 0.8997 sec, Total Search Time: 0.303608 sec, Total Time: 1.2033 sec
Points: 430,000, Insert Time: 0.0956 sec, Total Search Time: 1.328863 sec, Total Time: 1.4245 sec
Points: 440,000, Insert Time: 0.0886 sec, Total Search Time: 0.316772 sec, Total Time: 0.4054 sec
Points: 540,000, Insert Time: 0.8341 sec, Total Search Time: 1.762076 sec, Total Time: 2.5962 sec
Points: 1,540,000, Build Time: 2.3990 sec, Total

**-----VP Hybrid Log Run 3--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0180 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 10168.01it/s]

Total Search Time: 0.004123 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.3283 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 10787.82it/s]


...

Total Search Time: 0.524622 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 118361.19it/s]


Insert Time: 0.8480 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 33.32it/s]

Total Search Time: 3.117965 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0180 sec, Total Search Time: 0.004123 sec, Total Time: 0.0221 sec
Points: 110,000, Build Time: 0.3283 sec, Total Search Time: 0.003843 sec, Total Time: 0.3321 sec
Points: 210,000, Insert Time: 0.8573 sec, Total Search Time: 0.092994 sec, Total Time: 0.9503 sec
Points: 220,000, Insert Time: 0.0866 sec, Total Search Time: 0.448115 sec, Total Time: 0.5347 sec
Points: 320,000, Insert Time: 0.8282 sec, Total Search Time: 4.174342 sec, Total Time: 5.0026 sec
Points: 420,000, Insert Time: 0.8236 sec, Total Search Time: 0.291519 sec, Total Time: 1.1151 sec
Points: 430,000, Insert Time: 0.0856 sec, Total Search Time: 1.308673 sec, Total Time: 1.3942 sec
Points: 440,000, Insert Time: 0.0881 sec, Total Search Time: 0.315741 sec, Total Time: 0.4038 sec
Points: 540,000, Insert Time: 0.8497 sec, Total Search Time: 1.775426 sec, Total Time: 2.6251 sec
Points: 1,540,000, Build Time: 2.2659 sec, Total

**------VP Hybrid Log Analysis-------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 36.8260',
    'Total Time:: 38.3598',
    'Total Time:: 37.0585'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 37.4148 sec, Std Dev: 0.8266 sec, CV: 2.21%


--------------

**-----VP Hybrid Log Ratio Run 1-------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0393 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 14319.92it/s]


Total Search Time: 0.003362 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.0994 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 10877.34it/s]


Total Search Time: 0.004296 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.517519 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 114414.69it/s]


Insert Time: 0.8766 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 33.72it/s]

Total Search Time: 3.071066 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0393 sec, Total Search Time: 0.003362 sec, Total Time: 0.0427 sec
Points: 110,000, Build Time: 0.0994 sec, Total Search Time: 0.004296 sec, Total Time: 0.1037 sec
Points: 210,000, Insert Time: 0.8306 sec, Total Search Time: 0.092793 sec, Total Time: 0.9234 sec
Points: 220,000, Insert Time: 0.0848 sec, Total Search Time: 0.426855 sec, Total Time: 0.5116 sec
Points: 320,000, Insert Time: 0.8200 sec, Total Search Time: 4.081214 sec, Total Time: 4.9012 sec
Points: 420,000, Insert Time: 0.8254 sec, Total Search Time: 0.295275 sec, Total Time: 1.1207 sec
Points: 430,000, Insert Time: 0.0858 sec, Total Search Time: 1.287802 sec, Total Time: 1.3736 sec
Points: 440,000, Insert Time: 0.0969 sec, Total Search Time: 0.315138 sec, Total Time: 0.4120 sec
Points: 540,000, Insert Time: 0.8546 sec, Total Search Time: 1.741871 sec, Total Time: 2.5965 sec
Points: 1,540,000, Build Time: 2.3560 sec, Total

**------VP Hybrid Log Ratio Run 2---------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0166 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 14160.38it/s]


Total Search Time: 0.003030 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1356 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 10352.47it/s]


Total Search Time: 0.004762 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.517470 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 91497.60it/s]


Insert Time: 1.0950 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 39.75it/s]

Total Search Time: 2.622413 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0166 sec, Total Search Time: 0.003030 sec, Total Time: 0.0196 sec
Points: 110,000, Build Time: 0.1356 sec, Total Search Time: 0.004762 sec, Total Time: 0.1403 sec
Points: 210,000, Insert Time: 0.8687 sec, Total Search Time: 0.092290 sec, Total Time: 0.9610 sec
Points: 220,000, Insert Time: 0.0847 sec, Total Search Time: 0.444183 sec, Total Time: 0.5289 sec
Points: 320,000, Insert Time: 0.8283 sec, Total Search Time: 4.117248 sec, Total Time: 4.9455 sec
Points: 420,000, Insert Time: 0.8361 sec, Total Search Time: 0.295537 sec, Total Time: 1.1317 sec
Points: 430,000, Insert Time: 0.0856 sec, Total Search Time: 1.317413 sec, Total Time: 1.4030 sec
Points: 440,000, Insert Time: 0.0855 sec, Total Search Time: 0.322935 sec, Total Time: 0.4085 sec
Points: 540,000, Insert Time: 0.8188 sec, Total Search Time: 1.759627 sec, Total Time: 2.5784 sec
Points: 1,540,000, Build Time: 2.4347 sec, Total

**------VP Hybrid Log Ratio Run 3----------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0101 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 11007.23it/s]


Total Search Time: 0.003773 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1464 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 8340.23it/s]


Total Search Time: 0.004815 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.510684 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 120911.92it/s]


Insert Time: 0.8290 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 33.22it/s]

Total Search Time: 3.116211 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0101 sec, Total Search Time: 0.003773 sec, Total Time: 0.0139 sec
Points: 110,000, Build Time: 0.1464 sec, Total Search Time: 0.004815 sec, Total Time: 0.1512 sec
Points: 210,000, Insert Time: 0.8333 sec, Total Search Time: 0.108007 sec, Total Time: 0.9413 sec
Points: 220,000, Insert Time: 0.0970 sec, Total Search Time: 0.437363 sec, Total Time: 0.5344 sec
Points: 320,000, Insert Time: 0.8695 sec, Total Search Time: 4.134325 sec, Total Time: 5.0039 sec
Points: 420,000, Insert Time: 0.8257 sec, Total Search Time: 0.295537 sec, Total Time: 1.1212 sec
Points: 430,000, Insert Time: 0.0849 sec, Total Search Time: 1.308305 sec, Total Time: 1.3932 sec
Points: 440,000, Insert Time: 0.0853 sec, Total Search Time: 0.306796 sec, Total Time: 0.3921 sec
Points: 540,000, Insert Time: 0.8760 sec, Total Search Time: 1.752152 sec, Total Time: 2.6282 sec
Points: 1,540,000, Build Time: 2.4023 sec, Total

**------VP Hybrid Log Ratio Analysis------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 36.5292',
    'Total Time:: 38.0209',
    'Total Time:: 36.8019'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 37.1173 sec, Std Dev: 0.7943 sec, CV: 2.14%


----------

--------------

**-------VP Tree Epsilon Dynamic-------**

In [ ]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous point storage
        if n == 2*self.leaf_size:
            self.leaf_size = n

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
#*------------------------------------------------------------------------------------------

---------------

**------VP Dyn sqrt run 1------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0102 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 9336.24it/s]


Total Search Time: 0.003863 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1156 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 645.84it/s]


Total Search Time: 0.034032 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.503335 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 115739.23it/s]


Insert Time: 0.8670 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 52.70it/s]

Total Search Time: 2.048921 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0102 sec, Total Search Time: 0.003863 sec, Total Time: 0.0141 sec
Points: 110,000, Build Time: 0.1156 sec, Total Search Time: 0.034032 sec, Total Time: 0.1497 sec
Points: 210,000, Insert Time: 0.8834 sec, Total Search Time: 0.083647 sec, Total Time: 0.9671 sec
Points: 220,000, Insert Time: 0.0859 sec, Total Search Time: 0.347094 sec, Total Time: 0.4330 sec
Points: 320,000, Build Time: 0.5069 sec, Total Search Time: 0.038230 sec, Total Time: 0.5451 sec
Points: 420,000, Insert Time: 0.8446 sec, Total Search Time: 0.085471 sec, Total Time: 0.9301 sec
Points: 430,000, Insert Time: 0.0841 sec, Total Search Time: 0.356451 sec, Total Time: 0.4405 sec
Points: 440,000, Insert Time: 0.0849 sec, Total Search Time: 0.113637 sec, Total Time: 0.1985 sec
Points: 540,000, Insert Time: 0.8273 sec, Total Search Time: 0.723222 sec, Total Time: 1.5505 sec
Points: 1,540,000, Build Time: 2.4262 sec, Total 

**-----VP Dyn sqrt run 2-------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0344 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 15615.43it/s]


Total Search Time: 0.003539 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1063 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 10349.92it/s]


Total Search Time: 0.068035 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.468538 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 123482.06it/s]


Insert Time: 0.8121 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 51.99it/s]

Total Search Time: 2.067862 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0344 sec, Total Search Time: 0.003539 sec, Total Time: 0.0380 sec
Points: 110,000, Build Time: 0.1063 sec, Total Search Time: 0.068035 sec, Total Time: 0.1744 sec
Points: 210,000, Insert Time: 0.8614 sec, Total Search Time: 0.090498 sec, Total Time: 0.9519 sec
Points: 220,000, Insert Time: 0.0834 sec, Total Search Time: 0.361827 sec, Total Time: 0.4452 sec
Points: 320,000, Build Time: 0.3556 sec, Total Search Time: 0.014198 sec, Total Time: 0.3698 sec
Points: 420,000, Insert Time: 0.8197 sec, Total Search Time: 0.088125 sec, Total Time: 0.9078 sec
Points: 430,000, Insert Time: 0.0859 sec, Total Search Time: 0.367643 sec, Total Time: 0.4535 sec
Points: 440,000, Insert Time: 0.0859 sec, Total Search Time: 0.108074 sec, Total Time: 0.1939 sec
Points: 540,000, Insert Time: 0.8688 sec, Total Search Time: 0.768597 sec, Total Time: 1.6374 sec
Points: 1,540,000, Build Time: 2.3868 sec, Total 

**-----VP Dyn Sqrt run 3-------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0235 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 55260.92it/s]


Total Search Time: 0.002500 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1030 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 11389.83it/s]


Total Search Time: 0.057342 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.496121 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 119937.59it/s]


Insert Time: 0.8356 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 47.38it/s]

Total Search Time: 2.253901 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0235 sec, Total Search Time: 0.002500 sec, Total Time: 0.0260 sec
Points: 110,000, Build Time: 0.1030 sec, Total Search Time: 0.057342 sec, Total Time: 0.1604 sec
Points: 210,000, Insert Time: 0.8365 sec, Total Search Time: 0.090804 sec, Total Time: 0.9273 sec
Points: 220,000, Insert Time: 0.0995 sec, Total Search Time: 0.381295 sec, Total Time: 0.4808 sec
Points: 320,000, Build Time: 0.5249 sec, Total Search Time: 0.017555 sec, Total Time: 0.5424 sec
Points: 420,000, Insert Time: 0.8339 sec, Total Search Time: 0.088032 sec, Total Time: 0.9219 sec
Points: 430,000, Insert Time: 0.0840 sec, Total Search Time: 0.373030 sec, Total Time: 0.4570 sec
Points: 440,000, Insert Time: 0.0852 sec, Total Search Time: 0.100649 sec, Total Time: 0.1859 sec
Points: 540,000, Insert Time: 0.8286 sec, Total Search Time: 0.741381 sec, Total Time: 1.5700 sec
Points: 1,540,000, Build Time: 2.2549 sec, Total 

**-------Vp Dyn Sqrt Analysis--------**

In [ ]:
import re
import numpy as np


results = [
    'Total Time:: 26.1230',
    'Total Time:: 26.3649',
    'Total Time:: 26.9279'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 26.4719 sec, Std Dev: 0.4130 sec, CV: 1.56%


------------------

**-----VP Dynamic Log  Run 1------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0752 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 14665.40it/s]


Total Search Time: 0.004131 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1026 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 10878.76it/s]


Total Search Time: 0.003796 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.481042 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 119087.96it/s]


Insert Time: 0.8422 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 52.14it/s]

Total Search Time: 2.065513 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0752 sec, Total Search Time: 0.004131 sec, Total Time: 0.0793 sec
Points: 110,000, Build Time: 0.1026 sec, Total Search Time: 0.003796 sec, Total Time: 0.1064 sec
Points: 210,000, Insert Time: 0.8244 sec, Total Search Time: 0.082340 sec, Total Time: 0.9067 sec
Points: 220,000, Insert Time: 0.0845 sec, Total Search Time: 0.343339 sec, Total Time: 0.4278 sec
Points: 320,000, Insert Time: 0.8250 sec, Total Search Time: 3.104386 sec, Total Time: 3.9294 sec
Points: 420,000, Insert Time: 0.8211 sec, Total Search Time: 0.257505 sec, Total Time: 1.0786 sec
Points: 430,000, Insert Time: 0.0846 sec, Total Search Time: 1.000152 sec, Total Time: 1.0848 sec
Points: 440,000, Insert Time: 0.0859 sec, Total Search Time: 0.276375 sec, Total Time: 0.3623 sec
Points: 540,000, Insert Time: 0.8124 sec, Total Search Time: 1.424527 sec, Total Time: 2.2370 sec
Points: 1,540,000, Build Time: 2.3577 sec, Total

**--------VP Dynamic Log Run 2--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0193 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 19297.46it/s]


Total Search Time: 0.003380 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1722 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 11885.25it/s]


Total Search Time: 0.003579 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.456695 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 122719.84it/s]


Insert Time: 0.8171 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 51.94it/s]

Total Search Time: 2.073378 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0193 sec, Total Search Time: 0.003380 sec, Total Time: 0.0226 sec
Points: 110,000, Build Time: 0.1722 sec, Total Search Time: 0.003579 sec, Total Time: 0.1758 sec
Points: 210,000, Insert Time: 0.8190 sec, Total Search Time: 0.091904 sec, Total Time: 0.9109 sec
Points: 220,000, Insert Time: 0.0839 sec, Total Search Time: 0.360722 sec, Total Time: 0.4446 sec
Points: 320,000, Insert Time: 0.8346 sec, Total Search Time: 3.345963 sec, Total Time: 4.1805 sec
Points: 420,000, Insert Time: 0.8331 sec, Total Search Time: 0.276927 sec, Total Time: 1.1101 sec
Points: 430,000, Insert Time: 0.0844 sec, Total Search Time: 1.039572 sec, Total Time: 1.1239 sec
Points: 440,000, Insert Time: 0.0853 sec, Total Search Time: 0.283023 sec, Total Time: 0.3684 sec
Points: 540,000, Insert Time: 0.8172 sec, Total Search Time: 1.497450 sec, Total Time: 2.3147 sec
Points: 1,540,000, Build Time: 2.4629 sec, Total

**------VP Dynamic Log Run 3----------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0215 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 40021.98it/s]

Total Search Time: 0.003296 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.3456 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 12247.93it/s]


...

Total Search Time: 0.496213 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 123054.28it/s]


Insert Time: 0.8144 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 47.66it/s]

Total Search Time: 2.241504 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0215 sec, Total Search Time: 0.003296 sec, Total Time: 0.0248 sec
Points: 110,000, Build Time: 0.3456 sec, Total Search Time: 0.003374 sec, Total Time: 0.3489 sec
Points: 210,000, Insert Time: 0.8468 sec, Total Search Time: 0.090198 sec, Total Time: 0.9370 sec
Points: 220,000, Insert Time: 0.1084 sec, Total Search Time: 0.393209 sec, Total Time: 0.5016 sec
Points: 320,000, Insert Time: 0.8515 sec, Total Search Time: 3.431471 sec, Total Time: 4.2830 sec
Points: 420,000, Insert Time: 0.8316 sec, Total Search Time: 0.279901 sec, Total Time: 1.1115 sec
Points: 430,000, Insert Time: 0.0845 sec, Total Search Time: 1.111665 sec, Total Time: 1.1962 sec
Points: 440,000, Insert Time: 0.0851 sec, Total Search Time: 0.299293 sec, Total Time: 0.3844 sec
Points: 540,000, Insert Time: 0.8315 sec, Total Search Time: 1.513281 sec, Total Time: 2.3448 sec
Points: 1,540,000, Build Time: 2.3649 sec, Total

**-----VP Dyn Log Analysis------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 31.6623',
    'Total Time:: 30.8147',
    'Total Time:: 33.0654'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 31.8475 sec, Std Dev: 1.1367 sec, CV: 3.57%


-------------

**-----VP Dynamic Log Ratio Run 1-----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0228 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 8917.41it/s]


Total Search Time: 0.004410 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1071 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 8738.13it/s]


Total Search Time: 0.004904 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.474048 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 122318.08it/s]


Insert Time: 0.8204 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 52.75it/s]

Total Search Time: 2.038716 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0228 sec, Total Search Time: 0.004410 sec, Total Time: 0.0272 sec
Points: 110,000, Build Time: 0.1071 sec, Total Search Time: 0.004904 sec, Total Time: 0.1120 sec
Points: 210,000, Insert Time: 0.9416 sec, Total Search Time: 0.089887 sec, Total Time: 1.0314 sec
Points: 220,000, Insert Time: 0.0837 sec, Total Search Time: 0.365102 sec, Total Time: 0.4488 sec
Points: 320,000, Insert Time: 0.8397 sec, Total Search Time: 3.102353 sec, Total Time: 3.9420 sec
Points: 420,000, Insert Time: 0.8272 sec, Total Search Time: 0.269497 sec, Total Time: 1.0967 sec
Points: 430,000, Insert Time: 0.0850 sec, Total Search Time: 1.010524 sec, Total Time: 1.0956 sec
Points: 440,000, Insert Time: 0.0851 sec, Total Search Time: 0.302886 sec, Total Time: 0.3880 sec
Points: 540,000, Insert Time: 0.8168 sec, Total Search Time: 1.433117 sec, Total Time: 2.2499 sec
Points: 1,540,000, Build Time: 2.4152 sec, Total

**--------VP Dyn Log Ratio Run 2-------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0234 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 20702.39it/s]


Total Search Time: 0.003247 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.1613 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 13704.64it/s]


Total Search Time: 0.003483 sec

Batch 3: Adding 100,000 points


...

Total Search Time: 0.458368 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 117599.57it/s]


Insert Time: 0.8521 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 51.52it/s]

Total Search Time: 2.089796 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0234 sec, Total Search Time: 0.003247 sec, Total Time: 0.0266 sec
Points: 110,000, Build Time: 0.1613 sec, Total Search Time: 0.003483 sec, Total Time: 0.1648 sec
Points: 210,000, Insert Time: 0.8262 sec, Total Search Time: 0.086450 sec, Total Time: 0.9126 sec
Points: 220,000, Insert Time: 0.0841 sec, Total Search Time: 0.349192 sec, Total Time: 0.4333 sec
Points: 320,000, Insert Time: 0.8262 sec, Total Search Time: 3.320333 sec, Total Time: 4.1465 sec
Points: 420,000, Insert Time: 0.8227 sec, Total Search Time: 0.274022 sec, Total Time: 1.0967 sec
Points: 430,000, Insert Time: 0.0838 sec, Total Search Time: 1.068975 sec, Total Time: 1.1527 sec
Points: 440,000, Insert Time: 0.0848 sec, Total Search Time: 0.287501 sec, Total Time: 0.3723 sec
Points: 540,000, Insert Time: 0.8370 sec, Total Search Time: 1.501885 sec, Total Time: 2.3389 sec
Points: 1,540,000, Build Time: 2.4738 sec, Total

**-------VP Dyn Log Ratio Run 3--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0260 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 47554.47it/s]

Total Search Time: 0.002385 sec

Batch 2: Adding 100,000 points


Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.3413 sec


Querying batch 2: 100%|██████████| 20/20 [00:00<00:00, 12043.95it/s]


...

Total Search Time: 0.516142 sec

Batch 20: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 117606.86it/s]


Insert Time: 0.8537 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 47.47it/s]

Total Search Time: 2.251394 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0260 sec, Total Search Time: 0.002385 sec, Total Time: 0.0284 sec
Points: 110,000, Build Time: 0.3413 sec, Total Search Time: 0.003812 sec, Total Time: 0.3451 sec
Points: 210,000, Insert Time: 0.8214 sec, Total Search Time: 0.090065 sec, Total Time: 0.9114 sec
Points: 220,000, Insert Time: 0.0835 sec, Total Search Time: 0.377194 sec, Total Time: 0.4607 sec
Points: 320,000, Insert Time: 0.8209 sec, Total Search Time: 3.460693 sec, Total Time: 4.2816 sec
Points: 420,000, Insert Time: 0.8810 sec, Total Search Time: 0.283610 sec, Total Time: 1.1646 sec
Points: 430,000, Insert Time: 0.0882 sec, Total Search Time: 1.093224 sec, Total Time: 1.1814 sec
Points: 440,000, Insert Time: 0.0848 sec, Total Search Time: 0.303971 sec, Total Time: 0.3888 sec
Points: 540,000, Insert Time: 0.8376 sec, Total Search Time: 1.479252 sec, Total Time: 2.3169 sec
Points: 1,540,000, Build Time: 2.2196 sec, Total

**------VP Dyn Log Ratio Analysis------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 30.8468',
    'Total Time:: 31.8391',
    'Total Time:: 32.8968'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 31.8609 sec, Std Dev: 1.0252 sec, CV: 3.22%
